In [4]:
import pandas as pd
import utils
import plotly.graph_objects as go

pair = "EUR_GBP"
granularity = "H4"

MOM_WINDOW = 3

df = pd.read_pickle(utils.get_his_data_filename(pair, granularity))
non_cols = ['time', 'volume']
mod_cols = [x for x in df.columns if x not in non_cols]
df[mod_cols] = df[mod_cols].apply(pd.to_numeric)
df.describe()

,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
count,4671.000000,4671.000000,4671.000000,4671.000000,4671.000000,4671.000000,4671.000000,4671.000000,4671.000000,4671.000000,4671.000000,4671.000000,4671.000000
mean,3692.906872,0.883920,0.885112,0.882744,0.883919,0.883753,0.885001,0.882582,0.883770,0.884087,0.885267,0.882858,0.884069
std,3224.338646,0.019359,0.019494,0.019244,0.019359,0.019353,0.019489,0.019234,0.019352,0.019367,0.019501,0.019249,0.019368
min,2.000000,0.828420,0.830800,0.827690,0.828400,0.828300,0.830700,0.827580,0.828280,0.828530,0.830870,0.827800,0.828510
25%,1331.000000,0.872385,0.873680,0.871275,0.872370,0.872260,0.873560,0.871100,0.872245,0.872550,0.873845,0.871380,0.872560
50%,2998.000000,0.886040,0.887140,0.884810,0.886020,0.885890,0.887050,0.884710,0.885900,0.886190,0.887260,0.884900,0.886170
75%,5091.000000,0.898080,0.899280,0.896940,0.898080,0.897955,0.899195,0.896735,0.897930,0.898275,0.899490,0.897055,0.898265
max,36032.000000,0.947710,0.950000,0.942120,0.947680,0.947460,0.949840,0.941830,0.947460,0.947960,0.950280,0.942370,0.947890


In [5]:
df_mom = df[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c']].copy()
df_mom['DIR'] = df_mom.mid_c - df_mom.mid_o
df_mom['DIR'] = df_mom['DIR'].apply(lambda x: 1 if x >= 0 else -1)

for i in range(1, MOM_WINDOW+1):
    df_mom[f'DIR_PREV_{i}'] = df_mom.DIR.shift(i)
        
df_mom.dropna(inplace=True)
df_mom.head(10)

,time,mid_o,mid_h,mid_l,mid_c,DIR,DIR_PREV_1,DIR_PREV_2,DIR_PREV_3
3,2018-01-02 10:00:00+00:00,0.89094,0.89184,0.88914,0.88936,-1,1.0,1.0,1.0
4,2018-01-02 14:00:00+00:00,0.88938,0.88954,0.88592,0.88668,-1,-1.0,1.0,1.0
5,2018-01-02 18:00:00+00:00,0.88665,0.88734,0.88592,0.88734,1,-1.0,-1.0,1.0
6,2018-01-02 22:00:00+00:00,0.88740,0.88742,0.88596,0.88607,-1,1.0,-1.0,-1.0
7,2018-01-03 02:00:00+00:00,0.88609,0.88652,0.88580,0.88624,1,-1.0,1.0,-1.0
8,2018-01-03 06:00:00+00:00,0.88624,0.88647,0.88483,0.88611,-1,1.0,-1.0,1.0
9,2018-01-03 10:00:00+00:00,0.88608,0.88748,0.88494,0.88725,1,-1.0,1.0,-1.0
10,2018-01-03 14:00:00+00:00,0.88723,0.89002,0.88704,0.88917,1,1.0,-1.0,1.0
11,2018-01-03 18:00:00+00:00,0.88920,0.88973,0.88868,0.88888,-1,1.0,1.0,-1.0
12,2018-01-03 22:00:00+00:00,0.88902,0.88916,0.88851,0.88870,-1,-1.0,1.0,1.0


In [6]:
def is_trade(row, window):   
    signals = [ row[f'DIR_PREV_{i}'] for i in range(1, window+1)]
    if all(signal > 0 for signal in signals):
        return  -1
    elif all(signal < 0 for signal in signals):
        return 1
    return 0

def remove_duplicate_trades(row):
    if row.IS_TRADE_PREV == row.IS_TRADE:
        return 0
    else:
        return row.IS_TRADE
    

In [7]:
df_mom['IS_TRADE'] = df_mom.apply(lambda x : is_trade(x, 3), axis=1)
df_mom['IS_TRADE_PREV'] = df_mom.IS_TRADE.shift(1)
df_mom['IS_TRADE'] = df_mom.apply(remove_duplicate_trades, axis=1)
df_mom = df_mom.drop(['IS_TRADE_PREV'], axis=1)
df_mom.head(100)

,time,mid_o,mid_h,mid_l,mid_c,DIR,DIR_PREV_1,DIR_PREV_2,DIR_PREV_3,IS_TRADE
3,2018-01-02 10:00:00+00:00,0.89094,0.89184,0.88914,0.88936,-1,1.0,1.0,1.0,-1
4,2018-01-02 14:00:00+00:00,0.88938,0.88954,0.88592,0.88668,-1,-1.0,1.0,1.0,0
5,2018-01-02 18:00:00+00:00,0.88665,0.88734,0.88592,0.88734,1,-1.0,-1.0,1.0,0
6,2018-01-02 22:00:00+00:00,0.88740,0.88742,0.88596,0.88607,-1,1.0,-1.0,-1.0,0
7,2018-01-03 02:00:00+00:00,0.88609,0.88652,0.88580,0.88624,1,-1.0,1.0,-1.0,0
...,...,...,...,...,...,...,...,...,...,...
98,2018-01-24 06:00:00+00:00,0.87826,0.87841,0.87498,0.87537,-1,1.0,-1.0,1.0,0
99,2018-01-24 10:00:00+00:00,0.87539,0.87597,0.87106,0.87195,-1,-1.0,1.0,-1.0,0
100,2018-01-24 14:00:00+00:00,0.87197,0.87303,0.87003,0.87292,1,-1.0,-1.0,1.0,0
101,2018-01-24 18:00:00+00:00,0.87294,0.87310,0.86997,0.87135,-1,1.0,-1.0,-1.0,0


In [10]:
import instrument
i_pair = instrument.Instrument.get_instruments_dict()[pair]

In [11]:
df_trades = df_mom[df_mom.IS_TRADE != 0].copy()
df_trades["DELTA"] = (df_trades.mid_c.diff() / i_pair.pipLocation).shift(-1)
df_trades["GAIN"] = df_trades["DELTA"] * df_trades["IS_TRADE"]

In [12]:
df_trades.head()

,time,mid_o,mid_h,mid_l,mid_c,DIR,DIR_PREV_1,DIR_PREV_2,DIR_PREV_3,IS_TRADE,DELTA,GAIN
3,2018-01-02 10:00:00+00:00,0.89094,0.89184,0.88914,0.88936,-1,1.0,1.0,1.0,-1,9.2,-9.2
20,2018-01-05 06:00:00+00:00,0.88954,0.89076,0.88909,0.89028,1,-1.0,-1.0,-1.0,1,-32.8,-32.8
24,2018-01-07 22:00:00+00:00,0.88698,0.88758,0.88653,0.88700,1,-1.0,-1.0,-1.0,1,-36.4,-36.4
28,2018-01-08 14:00:00+00:00,0.88384,0.88427,0.88152,0.88336,-1,-1.0,-1.0,-1.0,1,62.5,62.5
45,2018-01-11 10:00:00+00:00,0.88584,0.89124,0.88496,0.88961,1,1.0,1.0,1.0,-1,-21.7,21.7


In [13]:
df_trades.GAIN.sum()

806.2000000000191